In [1]:
import pandas as pd 

column_names = (
    ["id", "cycle"] +
    ["setting1", "setting2", "setting3"] +
    [f"sensor{i}" for i in range(1, 22)]
)

df = pd.read_csv("../model_training/data/train_FD001.txt", sep=r"\s+", header=None, names=column_names)
df.head()

,id,cycle,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [2]:
import numpy as np 
import pandas as pd 

class DataSource:
    def __init__(self, df, feature_cols):
        self.feature_cols = feature_cols

        self.engines = {}
        for engine_id, g in df.groupby('id'):
            self.engines[engine_id] = g.sort_values('cycle').reset_index(drop=True)

        self.current_idx = {engine_id: 0 for engine_id in self.engines}

    def step(self):
        events = []

        for engine_id, df_engine in self.engines.items():
            idx = self.current_idx[engine_id]

            if idx >= len(df_engine):
                self.current_idx[engine_id] = 0
                idx = 0
            
            row = df_engine.iloc[idx]

            event = {
                "engine_id" : engine_id,
                "cycle" : int(row['cycle']),
                "features" : row[self.feature_cols].values.astype(np.float32)
            }

            events.append(event)

            self.current_idx[engine_id] += 1
        
        return events
    
    def run(self, n_steps=1, verbose=True):
        for t in range(n_steps):
            events = self.step()
            if verbose:
                for e in events:
                    print(e)

In [3]:
non_feature_cols = ["id", "cycle", "RUL"]
feature_cols = [c for c in df.columns if c not in non_feature_cols]

source = DataSource(df, feature_cols)

source.run()
source.run()

{'engine_id': 1, 'cycle': 1, 'features': array([-7.00000e-04, -4.00000e-04,  1.00000e+02,  5.18670e+02,
        6.41820e+02,  1.58970e+03,  1.40060e+03,  1.46200e+01,
        2.16100e+01,  5.54360e+02,  2.38806e+03,  9.04619e+03,
        1.30000e+00,  4.74700e+01,  5.21660e+02,  2.38802e+03,
        8.13862e+03,  8.41950e+00,  3.00000e-02,  3.92000e+02,
        2.38800e+03,  1.00000e+02,  3.90600e+01,  2.34190e+01],
      dtype=float32)}
{'engine_id': 2, 'cycle': 1, 'features': array([-1.80000e-03,  6.00000e-04,  1.00000e+02,  5.18670e+02,
        6.41890e+02,  1.58384e+03,  1.39128e+03,  1.46200e+01,
        2.16000e+01,  5.54530e+02,  2.38801e+03,  9.05472e+03,
        1.30000e+00,  4.69300e+01,  5.22330e+02,  2.38806e+03,
        8.13772e+03,  8.39050e+00,  3.00000e-02,  3.91000e+02,
        2.38800e+03,  1.00000e+02,  3.89400e+01,  2.34585e+01],
      dtype=float32)}
{'engine_id': 3, 'cycle': 1, 'features': array([8.00000e-04, 5.00000e-04, 1.00000e+02, 5.18670e+02, 6.42040e+02,
   

In [4]:
import sys, os
sys.path.append(os.path.abspath(".."))

import pandas as pd 
import numpy as np 

from inference.data_source import DataSource
from inference.engine_state import EngineState
from inference.engine_manager import EngineManager

In [5]:
df = pd.read_csv(
    "../model_training/data/train_FD001.txt",
    sep=r"\s+",
    header=None
)

df.columns = (
    ["id", "cycle"] +
    ["setting1", "setting2", "setting3"] +
    [f"sensor{i}" for i in range(1, 22)]
)

df = df[df["id"].isin([1, 2, 3])]
df = df.sort_values(["id", "cycle"]).reset_index(drop=True)

df.head()

,id,cycle,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [6]:
non_feature_cols = ["id", "cycle"]
feature_cols = [c for c in df.columns if c not in non_feature_cols]

In [7]:
source = DataSource(df, feature_cols)

for t in range(3):
    events = source.step()
    for e in events:
        print(
            e['engine_id'],
            e['cycle'],
            e['features'].shape
        )

1 1 (24,)
2 1 (24,)
3 1 (24,)
1 2 (24,)
2 2 (24,)
3 2 (24,)
1 3 (24,)
2 3 (24,)
3 3 (24,)


In [10]:
engine_state = EngineState(
    engine_id=1,
    window_size=5,
    num_features=len(feature_cols)
    
)


for i in range(6):
    fake_event = {
        "engine_id": 1,
        "cycle": i + 1,
        "features": np.random.randn(len(feature_cols))
    }

    engine_state.add_event(fake_event)

    print(
        f"Step {i+1} | ready={engine_state.is_ready()} | "
        f"buffer_len={len(engine_state.buffer)}"
    )

Step 1 | ready=False | buffer_len=1
Step 2 | ready=False | buffer_len=2
Step 3 | ready=False | buffer_len=3
Step 4 | ready=False | buffer_len=4
Step 5 | ready=True | buffer_len=5
Step 6 | ready=True | buffer_len=5


In [11]:
class DummyModel:
    def eval(self): pass
    def __call__(self, x):
        return torch.tensor([[42.0]])
    
class DummyScaler:
    def transform(self, x): return x

manager = EngineManager(
    model=DummyModel(),
    scaler=DummyScaler(),
    feature_cols=feature_cols,
    window_size=5
)

In [12]:
import torch

for t in range(10):
    events = source.step()
    preds = manager.process_events(events)

    print(f"\nTICK {t}")
    print("Predictions:", preds)


TICK 0
Predictions: []

TICK 1
Predictions: []

TICK 2
Predictions: []

TICK 3
Predictions: []

TICK 4
Predictions: [{'engine_id': 1, 'cycle': 8, 'rul_prediction': 42.0}, {'engine_id': 2, 'cycle': 8, 'rul_prediction': 42.0}, {'engine_id': 3, 'cycle': 8, 'rul_prediction': 42.0}]

TICK 5
Predictions: [{'engine_id': 1, 'cycle': 9, 'rul_prediction': 42.0}, {'engine_id': 2, 'cycle': 9, 'rul_prediction': 42.0}, {'engine_id': 3, 'cycle': 9, 'rul_prediction': 42.0}]

TICK 6
Predictions: [{'engine_id': 1, 'cycle': 10, 'rul_prediction': 42.0}, {'engine_id': 2, 'cycle': 10, 'rul_prediction': 42.0}, {'engine_id': 3, 'cycle': 10, 'rul_prediction': 42.0}]

TICK 7
Predictions: [{'engine_id': 1, 'cycle': 11, 'rul_prediction': 42.0}, {'engine_id': 2, 'cycle': 11, 'rul_prediction': 42.0}, {'engine_id': 3, 'cycle': 11, 'rul_prediction': 42.0}]

TICK 8
Predictions: [{'engine_id': 1, 'cycle': 12, 'rul_prediction': 42.0}, {'engine_id': 2, 'cycle': 12, 'rul_prediction': 42.0}, {'engine_id': 3, 'cycle': 12, 